In [1]:
from flightplotting import plot_regions
from flightdata import Flight, State
import numpy as np

from parsing import make_controller_field, get_last_log


np.set_printoptions(suppress=True)

fl = Flight.from_log(get_last_log(), 
    ['TSPD', 'TRAN', 'TRAL', 'TPIT', 'TPIG', 'TYAW', 'TINF'], 
    TSPD = make_controller_field('tspd'),
    TRAN = make_controller_field('tran'),
    TRAL = make_controller_field('tral'),
    TPIT = make_controller_field('tpit'),
    TPIG = make_controller_field('tpig'),
    TYAW = make_controller_field('tyaw'),
    TINF = dict(
        tinf_id='id',
        tinf_cmd='cmd',
        tinf_loadfactor='loadfactor',
        tinf_stage='stage'
    )
)

In [2]:
st = State.from_flight(fl).label(
    turn=fl.tinf_id.fillna(0).astype(int).map(lambda x: f'turn_{x}'),
    stage=fl.tinf_stage.fillna(99).astype(int).map(lambda x: f'stage_{x}'),
)




In [3]:
st.data.stage.unique()

array(['stage_99', 'stage_0', 'stage_1', 'stage_2'], dtype=object)

In [4]:
from dataclasses import dataclass
import numpy as np
import pandas as pd

@dataclass
class FlSt:
    g: float
    arsp: float
    fl: Flight
    st: State

    def summary(self):
        return dict(
            target_g=self.g,
            current = np.mean(self.fl.battery_current),
            voltage = np.mean(self.fl.battery_voltage),
            power = np.mean(self.fl.battery_current * self.fl.battery_voltage),
            measured_g = np.mean(-self.st.dw / 9.81),
            airspeed = np.mean(self.fl.air_speed),
            altitude = np.mean(self.st.z),
            throttle = np.mean(self.fl.rcout_c3)
        )

flsts = []

for id in fl.data.tinf_id.unique():
    if not np.isnan(id):
        stt = st.get_label_subset(turn=f'turn_{int(id)}', stage='stage_2')[42:]
        flt = fl.slice_time_flight(slice(stt.t[0],stt.t[-1]))
        flsts.append(FlSt(
            np.round(flt.data.tinf_loadfactor.iloc[0], 2),
            np.round(flt.data.air_speed.mean(), 2),
            flt, stt
        ))


summarydf = pd.DataFrame([flst.summary() for flst in flsts])
summarydf['group'] = np.cumsum(np.abs(summarydf.airspeed.diff()) > 0.1)
flsts

[FlSt(g=1.2, arsp=14.49, fl=<flightdata.flight.flight.Flight object at 0x7f252d7fc9e0>, st=State Table(duration = 32.839999999999996))]

In [5]:
import plotly.graph_objects as go
from flightplotting import colour_from_scale


maxg = max(flst.g for flst in flsts)

fig = go.Figure()

for flst in flsts:
    _g = flst.g
    _fl = flst.fl
    _st = flst.st

    common = dict( line=dict(color=colour_from_scale(_g, maxg)), showlegend=False, name=f'g={_g}')

    fig.add_trace(go.Scatter(x=_fl.data.index, y=_fl.rcout_c3, **common))
    fig.add_trace(go.Scatter(x=_fl.data.index, y=_fl.air_speed, yaxis='y2',  **common))
    fig.add_trace(go.Scatter(x=_fl.data.index, y=-_st.acc.z / 9.81, yaxis='y3',  **common))
    fig.add_trace(go.Scatter(x=_fl.data.index, y=_st.z, yaxis='y4',  **common))


nplots = 4
domains = [(i/(nplots), (i+1)/(nplots) ) for i in range(nplots)]

fig.update_layout(
    xaxis=dict(title='time, s'),
    yaxis=dict(title='Throttle pwm, ms', domain=domains[0]),
    yaxis2=dict(title='Airspeed, m/s', domain=domains[1]),
    yaxis3=dict(title='Load Factor, g', domain=domains[2]),
    yaxis4=dict(title='altitude, m', domain=domains[3]),
    margin=dict(l=20, r=20, t=20, b=20),
    height=800
)
fig

In [6]:
import plotly.graph_objects as go
flst = flsts[0]
fig = go.Figure()
fig.add_trace(go.Scatter(y=flst.fl.data.tspd_Total, name='tspd_Total'))
fig.add_trace(go.Scatter(y=flst.fl.air_speed, name='airspeed', yaxis='y2'))
fig.add_trace(go.Scatter(y=flst.st.z, name='altitude', yaxis='y3'))
fig.update_layout(
    xaxis=dict(title='time, s'),
    yaxis=dict(title='control pwm, ms'),
    yaxis2=dict(title='airspeed, m/s', overlaying='y', anchor='free', autoshift=True, range=[10, 20]),
    yaxis3=dict(title='altitude, m', overlaying='y', anchor='free', autoshift=True, range=[90, 105]),
)
fig

In [10]:
#fl.tspd_Total
flsts[9].fl.tspd_Total

,tspd_Total
time_index,
0.00,4.655632
0.04,4.617305
0.08,4.525565
0.12,4.918297
0.16,4.889507
...,...
30.28,9.396283
30.32,9.979884
30.36,10.023740


In [24]:
summarydf
#for i, df in summarydf.groupby('group'):
#    px.line(df, x='measured_g', y='power').show()
#

,target_g,current,voltage,power,measured_g,airspeed,altitude,throttle,group
0,1.2,5.004790,12.529933,62.709419,1.199938,16.909062,98.890247,1100.095808,0
1,1.6,5.005168,12.529927,62.714102,1.600691,17.347075,91.763470,1100.103359,1
2,2.0,5.005175,12.529927,62.714185,1.751367,17.642495,82.799517,1100.157827,2
3,2.4,5.005298,12.529926,62.715715,1.760118,17.745788,75.132089,1100.133775,3
4,2.8,5.005333,12.529925,62.716153,1.786848,17.959443,70.619155,1100.108000,4
5,1.2,5.276867,12.526124,66.096999,1.199973,17.336355,98.859164,1105.560241,5
6,1.6,5.005096,12.529928,62.713204,1.599969,17.436858,92.175282,1100.101911,6
7,2.0,5.006491,12.529909,62.730587,1.749084,17.622313,82.787438,1100.190231,7
8,2.4,5.005844,12.529918,62.722525,1.771917,17.811724,74.970372,1100.146753,8
9,2.8,5.012617,12.529823,62.806889,1.802979,18.097243,71.659687,1100.248322,9


In [26]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=summarydf.measured_g, y=summarydf.throttle, name='power'))
fig.add_trace(go.Scatter(x=summarydf.measured_g, y=summarydf.airspeed, yaxis='y2', name='airspeed'))
fig.update_layout(
    xaxis=dict(title='Load Factor, g'),
    yaxis=dict(title='throttle pwm, ms'),
    yaxis2=dict(title='airspeed, m/s', overlaying='y', anchor='free', autoshift=True),
)

In [16]:
from flightplotting import plotsec

plotsec(flsts[1].st, scale=1, nmodels=10).show()